In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
from collections import Counter
from itertools import combinations, groupby

In [5]:
orders=pd.read_csv('C:/Users/manan/Desktop/Engineering Management/Market_Basket_Analysis/order_products__prior.csv')

In [6]:
products=pd.read_csv('C:/Users/manan/Desktop/Engineering Management/Market_Basket_Analysis/products.csv')

In [7]:
orders.shape

(32434489, 4)

In [8]:
orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [9]:
#using the order as a series
orders=orders.set_index('order_id')['product_id']

In [10]:
print("total no. of orders")
len(orders.index.unique())

total no. of orders


3214874

In [11]:
print("total unique products sold")
len(orders.value_counts())

total unique products sold


49677

In [12]:
orders.head(10)

order_id
2    33120
2    28985
2     9327
2    45918
2    30035
2    17794
2    40141
2     1819
2    43668
3    33754
Name: product_id, dtype: int64

In [13]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [14]:
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")
    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))

#generating the item pairs
def get_item_pairs(order_item):
    order_item=order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda X:X[0]  ):
        item_list= [item[1] for item in order_object]
        
        for item_pair in combinations(item_list,2):
            yield item_pair
    
def merge_item_stats(item_pairs,item_stats):
    return (item_pairs.merge(item_stats.rename(columns={'freq':'freqA','support':'supportA'}),left_on='item_A', right_index=True).merge(item_stats.rename(columns={'freq':'freqB','support':'supportB'}),left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]

In [15]:
#calculating the association rules:
def association_rule(orders_product,min_support):
    print("starting/total product: {:22}".format(len(orders)))
    #item frequency and support
    item_stats= freq(orders_product).to_frame('freq')
    #unique count of orders
    #
    item_stats['support']= item_stats['freq']/order_count(orders_product)*100
    
    #filtering items below min support
    qualifying_product=item_stats[item_stats['support']>=min_support].index
    #taking only the product order combination with qualifying product with min support
    orders_product=orders_product[orders_product.isin(qualifying_product)]
    
    print("products with support >={}:{:15d}".format(min_support, len(qualifying_product)))
    print("Remainig products: {:21}".format(len(orders_product)))
#     print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    
    #filtering out order items with less than 2 products
    order_size= freq(orders_product.index)
    qualifying_products=order_size[order_size>=2].index
    orders_product=orders_product[orders_product.index.isin(qualifying_products)]
    
    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_products)))
    print("Remaining order_product: {:21d}".format(len(orders_product)))
    
    #recalculating item/product frequenchy and support
    item_stats = freq(orders_product).to_frame('freq')
    item_stats['support']= item_stats['freq']/order_count(orders_product)*100
    
    #retrieving item/product pairs
    item_pair_gen= get_item_pairs(orders_product)
    
    #calculating item pairs frequency and support
    item_pairs= freq(item_pair_gen).to_frame('freqAB')
    item_pairs['supportAB']= item_pairs['freqAB']/len(qualifying_products)*100
    
    print("Product pairs: {:31d}".format(len(item_pairs)))
    
    #Filter out item pairs with support below min support
    item_pairs= item_pairs[item_pairs['supportAB']>=min_support]
    print('item pair with support >= {}:{:10d}\n'.format(min_support,len(item_pairs)))
    
    #Association rules and matrices
    item_pairs= item_pairs.reset_index().rename(columns={'level_0':'item_A','level_1':'item_B'})
    item_pairs= merge_item_stats(item_pairs,item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    #returning association rules sorted by lift in descending order
    return item_pairs.sort_values('lift',ascending=False)


In [19]:
%%time
print(time.time())
rules=association_rule(orders,0.01)
print(time.time())

1574644569.5588677
starting/total product:               32434489
products with support >=0.01:          10906
Remainig products:              29843570
Remaining orders with 2+ items:     3013325
Remaining order_product:              29662716


C:\Users\manan\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


Product pairs:                        30622410
item pair with support >= 0.01:     48751

1574645113.2048693
Wall time: 9min 3s


In [21]:
item_name   = products.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)
display(rules_final)

,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,Organic Strawberry Chia Lowfat 2% Cottage Cheese,Organic Cottage Cheese Blueberry Acai Chia,306,0.010155,1163,0.038595,839,0.027843,0.263113,0.364720,9.449868
1,Grain Free Chicken Formula Cat Food,Grain Free Turkey Formula Cat Food,318,0.010553,1809,0.060033,879,0.029170,0.175788,0.361775,6.026229
3,Organic Fruit Yogurt Smoothie Mixed Berry,Apple Blueberry Fruit Yogurt Smoothie,349,0.011582,1518,0.050376,1249,0.041449,0.229908,0.279424,5.546732
9,Nonfat Strawberry With Fruit On The Bottom Gre...,"0% Greek, Blueberry on the Bottom Yogurt",409,0.013573,1666,0.055288,1391,0.046162,0.245498,0.294033,5.318230
10,Organic Grapefruit Ginger Sparkling Yerba Mate,Cranberry Pomegranate Sparkling Yerba Mate,351,0.011648,1731,0.057445,1149,0.038131,0.202773,0.305483,5.317849
...,...,...,...,...,...,...,...,...,...,...,...
7271,Organic Strawberries,Strawberries,640,0.021239,263416,8.741706,141805,4.705931,0.002430,0.004513,0.000516
6763,Organic Hass Avocado,Organic Avocado,464,0.015398,212785,7.061469,176241,5.848722,0.002181,0.002633,0.000373
4387,Organic Avocado,Organic Hass Avocado,443,0.014701,176241,5.848722,212785,7.061469,0.002514,0.002082,0.000356
2596,Banana,Bag of Organic Bananas,654,0.021704,470096,15.600574,376367,12.490090,0.001391,0.001738,0.000111
